In [2]:
import azure.cosmos
import os
import json
from uuid import uuid4
from tabulate import tabulate

cosmos_endpoint = os.getenv("AZURE_COSMOS_ENDPOINT")
cosmos_key = os.getenv("AZURE_COSMOS_KEY")

cosmos_client = azure.cosmos.CosmosClient(cosmos_endpoint, cosmos_key)

database_id = "db"
container_id = "hotels"

database = cosmos_client.create_database_if_not_exists(id=database_id)
container = database.create_container_if_not_exists(id=container_id, partition_key=azure.cosmos.PartitionKey(path="/HotelId"))

In [ ]:
with open("hotels.json", encoding="utf-8") as f:
    hotels = json.load(f)
    for hotel in hotels:
        hotel["id"] = str(uuid4())
        container.upsert_item(hotel)

In [89]:
query_prompt = """
Write a Cosmos DB SQL Query for a container with data about hotels. It has the following JSON schema:
HotelId: Unique ID of the hotel,
HotelName: Name of the hotel,
Description: Description of the hotel,
Category: What type of hotel this is (e.g. boutique, budget),
Tags: String array containing description tags for the hotel,
ParkingIncluded: Whether or not parking is included in the hotel fee,
Rating: Rating of the hotel,
Address: Object containing StreetAddress, City, StateProvince, PostalCode, Country fields,
Location: Geo-location of the hotel,
Rooms: Array of objects containing a Description, Type, BaseRate, BedOptions, SleepsCount, SmokingAllowed, and Tags string array

The query needs to find the following information: {}. If you are returning aggregate functions, be sure to include VALUE before calling those functions

Output only the query and nothing else
"""

In [79]:
functions=[
    {
        "name": "query_hotels",
        "description": "Queries a Cosmos DB database containing information about hotels",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type": "string",
                    "description": "Natural language question you want to query the database to answer"
                }
            },
            "required": ["query"]
        }
    },
    {
        "name": "book_hotel",
        "description": "Makes a hotel booking for a specific time range",
        "parameters": {
            "type": "object",
            "properties": {
                "hotelName": {
                    "type": "string",
                    "description": "Name of the hotel to book"
                },
                "startDate": {
                    "type": "string",
                    "description": "First day of the hotel booking in yyyy-MM-dd format"
                },
                "endDate": {
                    "type": "string",
                    "description": "Last day of the hotel booking in yyyy-MM-dd format"
                }
            }
        }
    }
]

In [84]:
system_message = {"role": "system", "content": "You are a helpful assistant helping the user learn about what hotels are available, the qualities and characteristics of the hotels, and booking them. If you are answering a question about a hotel, always include the hotel name in your answer. Be as concise as possible"}

In [90]:
from openai import AzureOpenAI

client = AzureOpenAI(
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2023-10-01-preview",
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
)

def run_agent(question):
    messages = []
    messages.append({"role": "user", "content": question})
    response = client.chat.completions.create(
        model=os.getenv("AZURE_OPENAI_CHATGPT_DEPLOYMENT"),
        messages=messages,
        functions=functions,
        function_call="auto"
    )

    response_message = response.choices[0].message
    if response_message.function_call:
        function_call = response.choices[0].message.function_call
        if function_call.name == "query_hotels":
            query = json.loads(function_call.arguments)["query"]
            query_response = client.chat.completions.create(
                model=os.getenv("AZURE_OPENAI_CHATGPT_DEPLOYMENT"),
                messages=[{"role": "user", "content": query_prompt.format(query)}],
            )
            cosmos_db_query = query_response.choices[0].message.content
            print("Generated Cosmos DB SQL Query", cosmos_db_query)
            try:
                items = container.query_items(
                    query=cosmos_db_query,
                    enable_cross_partition_query=True
                )
                # Extract the results as a list of dictionaries
                results = list(items)
                print(results)


            except Exception as e:
                print("Failed to generate SQL query!", e)
                results = None
        
            if results:
                # Extract column names from the first result
                columns = results[0].keys() if isinstance(results[0], dict) else {"value":"value"}

                # Convert the list of dictionaries into a list of lists for tabulate
                table_data = [[row[column] for column in columns] if isinstance(row, dict) else {"value": row} for row in results]

                markdown_table = tabulate(table_data, headers=columns, tablefmt="pipe")

                print(markdown_table)

                messages.append(
                    {
                        "role": response_message.role,
                        "function_call": {
                            "name": "query_hotels",
                            "arguments": response_message.function_call.arguments
                        },
                        "content": None
                    }
                )
                messages.append(
                    {
                        "role": "function",
                        "name": "query_hotels",
                        "content": markdown_table
                    }
                )

                second_response = client.chat.completions.create(
                    model=os.getenv("AZURE_OPENAI_CHATGPT_DEPLOYMENT"),
                    messages=messages
                )
                print(second_response.choices[0].message.content)
        elif function_call.name == "book_hotel":
            params = json.loads(function_call.arguments)
            hotel_name = params["hotelName"]
            start_date = params["startDate"]
            end_date = params["endDate"]
            print(f"Booking {hotel_name} from {start_date} to {end_date}")





In [91]:
run_agent("Find the average hotel rating")

Generated Cosmos DB SQL Query SELECT VALUE AVG(c.Rating) FROM c
[3.73]
|   value |
|--------:|
|    3.73 |
The average hotel rating is 3.73.


In [92]:
run_agent("What are the hotels are in New York?")

Generated Cosmos DB SQL Query SELECT * FROM Hotels h WHERE h.Address.City = "New York"
[{'HotelId': '1', 'HotelName': 'Secret Point Hotel', 'Description': "This classic hotel is ideally located on the main commercial artery of the city in the heart of New York. A few minutes away is Time's Square and the historic centre of the city, as well as other places of interest that make New York one of America's most attractive and cosmopolitan cities.", 'Description_fr': "Cet hôtel classique est idéalement situé sur la principale artère commerciale de la ville en plein cœur de New York. A quelques minutes se trouve la place du temps et le centre historique de la ville, ainsi que d'autres lieux d'intérêt qui font de New York l'une des villes les plus attractives et cosmopolites de l'Amérique.", 'Category': 'Boutique', 'Tags': ['view', 'air conditioning', 'concierge'], 'ParkingIncluded': False, 'LastRenovationDate': '2017-01-18T00:00:00Z', 'Rating': 3.6, 'Address': {'StreetAddress': '677 5th Ave

In [94]:
run_agent("Find the rates for rooms that sleep at least 2 in all hotels in New York")

Generated Cosmos DB SQL Query SELECT c.HotelName, r.Type, r.BaseRate
FROM c JOIN r IN c.Rooms
WHERE r.SleepsCount >= 2 AND c.Address.City = 'New York'
[{'HotelName': 'Secret Point Hotel', 'Type': 'Budget Room', 'BaseRate': 96.99}, {'HotelName': 'Secret Point Hotel', 'Type': 'Budget Room', 'BaseRate': 80.99}, {'HotelName': 'Secret Point Hotel', 'Type': 'Deluxe Room', 'BaseRate': 150.99}, {'HotelName': 'Secret Point Hotel', 'Type': 'Standard Room', 'BaseRate': 110.99}, {'HotelName': 'Secret Point Hotel', 'Type': 'Suite', 'BaseRate': 243.99}, {'HotelName': 'Secret Point Hotel', 'Type': 'Suite', 'BaseRate': 229.99}, {'HotelName': 'Secret Point Hotel', 'Type': 'Budget Room', 'BaseRate': 87.99}, {'HotelName': 'Secret Point Hotel', 'Type': 'Suite', 'BaseRate': 262.99}, {'HotelName': 'Secret Point Hotel', 'Type': 'Suite', 'BaseRate': 248.99}, {'HotelName': 'Secret Point Hotel', 'Type': 'Suite', 'BaseRate': 234.99}, {'HotelName': 'Secret Point Hotel', 'Type': 'Deluxe Room', 'BaseRate': 146.99},

In [95]:
run_agent("book the secret point hotel from 11-1-2023 to 11-4-2023")

Booking the secret point hotel from 2023-01-11 to 2023-04-11
